In [ ]:
!pip install imageio
!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

class SchwarzschildRayTracerBatch:
    def __init__(self, bh_mass=1.0, device=None):
        self.G = 1.0
        self.c = 1.0
        self.E = 1.0
        self.M = bh_mass
        self.rs = 2 * self.G * self.M / self.c**2
        self.b_max = 3 * np.sqrt(3) / 2 * self.rs
        self.device = device if device else (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

    def V(self, r, b):
        return (b**2 / (2 * r**2) - self.M * b**2 / r**3)

    def dV_dr(self, r, b):
        return (-b**2 / r**3 + 3 * self.M * b**2 / r**4)

    def system(self, y, b):
        r = y[:, 0]
        phi = y[:, 1]
        dr_dtau = y[:, 2]

        d2r_dtau2 = -self.dV_dr(r, b)
        dphi_dtau = b / r**2

        dydtau = torch.stack([dr_dtau, dphi_dtau, d2r_dtau2], dim=1)
        return dydtau

    def rkf45_step_batch(self, f, t, y, h, b):
        a = [0, 1/4, 3/8, 12/13, 1, 1/2]
        b_ = [
            [],
            [1/4],
            [3/32, 9/32],
            [1932/2197, -7200/2197, 7296/2197],
            [439/216, -8, 3680/513, -845/4104],
            [-8/27, 2, -3544/2565, 1859/4104, -11/40]
        ]
        c4 = [25/216, 0, 1408/2565, 2197/4104, -1/5, 0]
        c5 = [16/135, 0, 6656/12825, 28561/56430, -9/50, 2/55]

        k1 = f(y, b)
        k2 = f(y + h * (b_[1][0] * k1), b)
        k3 = f(y + h * (b_[2][0] * k1 + b_[2][1] * k2), b)
        k4 = f(y + h * (b_[3][0] * k1 + b_[3][1] * k2 + b_[3][2] * k3), b)
        k5 = f(y + h * (b_[4][0] * k1 + b_[4][1] * k2 + b_[4][2] * k3 + b_[4][3] * k4), b)
        k6 = f(y + h * (b_[5][0] * k1 + b_[5][1] * k2 + b_[5][2] * k3 + b_[5][3] * k4 + b_[5][4] * k5), b)

        y4 = y + h * (c4[0]*k1 + c4[2]*k3 + c4[3]*k4 + c4[4]*k5)
        y5 = y + h * (c5[0]*k1 + c5[2]*k3 + c5[3]*k4 + c5[4]*k5 + c5[5]*k6)

        error = torch.norm(y5 - y4, dim=1)
        return y5, error

    def solve_batch(self, b_array, r0=100.0, phi0=0.0, tau_span=(0, 140), h=0.1, tol=1e-5):
        b = torch.tensor(b_array, dtype=torch.float32, device=self.device)
        batch_size = b.shape[0]
        r0_t = torch.full((batch_size,), r0, dtype=torch.float32, device=self.device)
        phi0_t = torch.full((batch_size,), phi0, dtype=torch.float32, device=self.device)

        V0 = self.V(r0_t, b)
        dr0 = -torch.sqrt(torch.clamp(self.E**2 - 2 * V0, min=0.0))

        y = torch.stack([r0_t, phi0_t, dr0], dim=1)
        t = tau_span[0]
        tf = tau_span[1]

        ys = [y.cpu()]
        active = torch.ones(batch_size, dtype=torch.bool, device=self.device)

        while t < tf and active.any():
            y_next, error = self.rkf45_step_batch(self.system, t, y, h, b)
            y = torch.where(active.unsqueeze(1), y_next, y)
            active = active & (y[:, 0] > self.rs)
            ys.append(y.cpu())
            t += h

        return torch.stack(ys)

    def render(self, N_x=100, N_y=100, phi_deg=0, save_path=None):
        ancho, alto = N_x, N_y
        imagen = Image.new('RGB', (ancho, alto), color=(0,0,0))
        imagen2 = Image.new('RGB', (ancho, alto), color=(0,0,0))
        pixeles = imagen.load()
        pixeles2= imagen2.load()
        R = self.matriz_rotacion(phi_deg)

        xs = np.linspace(-1, 1, ancho)
        ys = np.linspace(-1, 1, alto)
        xv, yv = np.meshgrid(xs, ys)
        b_vals = self.b_max * np.sqrt(xv.flatten()**2 + yv.flatten()**2)
        mask = b_vals < self.b_max
        b_vals_filtered = b_vals[mask]

        sol_batch = self.solve_batch(b_vals_filtered, r0=100, tau_span=(0,140), h=0.1)
        rs = self.rs
        r_vals = sol_batch[:, :, 0].numpy()
        phi_vals = sol_batch[:, :, 1].numpy()

        idx = 0
        for i in range(ancho):
            for j in range(alto):
                b = self.b_max * np.sqrt(xv[j,i]**2 + yv[j,i]**2)
                if b >= self.b_max:
                    pixeles[i,j] = (0,0,0)
                    continue

                r_traj = r_vals[:, idx]
                phi_traj = phi_vals[:, idx]

                if r_traj[-1] > rs:
                    phi = phi_traj[-1]
                else:
                    phi = np.interp(rs, r_traj[-2:], phi_traj[-2:])

                theta = np.arctan2(yv[j,i], xv[j,i])
                x = rs * np.cos(theta) * np.sin(phi)
                y = rs * np.sin(theta) * np.sin(phi)
                z = -rs * np.cos(phi)
                x1,y1,z1 = np.dot([x,y,z], R)
                theta1 = np.arccos(z1 / rs)+np.pi/2
                phi1 = np.arctan2(y1, x1)+np.pi

                if ((int(phi1 / (2*np.pi) * 12) % 2) == 0) and ((int(theta1 / np.pi * 7) % 2) == 0):
                    color = (0, 0, 255)
                elif ((int(phi1 / (2*np.pi) * 12 + 1) % 2) == 0) and ((int(theta1 / np.pi * 7 + 1) % 2) == 0):
                    color = (0, 0, 255)
                else:
                    color = (0, 0, 0)

                pixeles[i, j] = color

                if (b<=rs):
                  phi0=np.arcsin(b/self.rs)
                  x = rs * np.cos(theta) * np.sin(phi)
                  y = rs * np.sin(theta) * np.sin(phi)
                  z = -rs * np.cos(phi)
                  x1,y1,z1 = np.dot([x,y,z], R)
                  theta1 = np.arccos(z1 / rs)+np.pi/2
                  phi1 = np.arctan2(y1, x1)+np.pi
                  if ((int(phi1 / (2*np.pi) * 12) % 2) == 0) and ((int(theta1 / np.pi * 7) % 2) == 0):
                      color2 = (255, 0, 0)
                  elif ((int(phi1 / (2*np.pi) * 12 + 1) % 2) == 0) and ((int(theta1 / np.pi * 7 + 1) % 2) == 0):
                      color2 = (255, 0, 0)
                  else:
                    color2 = (0, 0, 0)
                else:
                    color2 = (0, 0, 0)

                pixeles2[i, j] = color2

                idx += 1
        # Concatenar ambas imágenes (de manera horizontal)
        combined_image = Image.new('RGB', (2 * ancho, alto))
        combined_image.paste(imagen, (0, 0))
        combined_image.paste(imagen2, (ancho, 0))

        if save_path:
            combined_image.save(save_path)

        else:
          combined_image.save("Imagen_horizonte_sucesos.png")
          from google.colab import files
          files.download("Imagen_horizonte_sucesos.png")

    def matriz_rotacion(self, phi_deg):
        theta = np.deg2rad(0)
        phi = np.deg2rad(phi_deg)
        Rz = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta),  np.cos(theta), 0],
            [0, 0, 1]
        ])
        Ry = np.array([
            [np.cos(phi), 0, np.sin(phi)],
            [0, 1, 0],
            [-np.sin(phi), 0, np.cos(phi)]
        ])
        return Rz @ Ry
rt_batch = SchwarzschildRayTracerBatch(bh_mass=1.0)
rt_batch.render(N_x=500, N_y=500, phi_deg=50)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import imageio
from IPython.display import FileLink
from tqdm import tqdm


# Crear directorio de salida
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

# Crear instancia
rt_batch = SchwarzschildRayTracerBatch(bh_mass=1.0)

# Ángulos de rotación
phis = np.arange(0, 360, 10)

# Renderizar
for phi in tqdm(phis, desc="Renderizando imágenes"):
    filename = os.path.join(output_dir, f"frame_{phi:03d}.png")
    rt_batch.render(N_x=300, N_y=300, phi_deg=phi, save_path=filename)

# Crear video
video_filename = "rotacion.mp4"
with imageio.get_writer(video_filename, fps=10) as writer:
    for phi in phis:
        filename = os.path.join(output_dir, f"frame_{phi:03d}.png")
        writer.append_data(imageio.imread(filename))

print("✅ Video creado:", video_filename)
FileLink(video_filename)


Renderizando imágenes: 100%|██████████| 36/36 [03:25<00:00,  5.70s/it]
/tmp/ipython-input-25-2737085095.py:26: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  writer.append_data(imageio.imread(filename))
/tmp/ipython-input-25-2737085095.py:26: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  writer.append_data(imageio.imread(filename))


✅ Video creado: rotacion.mp4


/content/rotacion.mp4

In [ ]:
from google.colab import files
files.download("rotacion.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>